In [2]:
import arcpy, os, sys, shutil
from arcpy import env

In [22]:
env.overwriteOutput = True

In [4]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/network-analyst/build-network.htm
try:
    #Check out Network Analyst license if available. Fail if the Network Analyst license is not available.
    if arcpy.CheckExtension("network") == "Available":
        arcpy.CheckOutExtension("network")
    else:
        raise arcpy.ExecuteError("Network Analyst Extension license is not available.")
    
    #Set environment settings
    env.workspace = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources\Network_Analysis\Network_Analysis.gdb"
    
    #Set local variables
    inNetworkDataset = "Network_Ped_Bike/Network_ND"
    
    #Build the network dataset
    arcpy.BuildNetwork_na(inNetworkDataset)
    
    #If there are any build errors, they are recorded in a BuildErrors.txt file
    #present in the system temp directory. So copy this file to the directory
    #containing this script
    
    #First get the path to the system temp directory
    tempDir = os.environ.get("TEMP")
    if tempDir:
        shutil.copy2(os.path.join(tempDir,"BuildErrors.txt"),sys.path[0])
    
    print("Script completed successfully.")

except Exception as e:
    # If an error occurred, print line number and error message
    import traceback, sys
    tb = sys.exc_info()[2]
    print("An error occurred on line %i" % tb.tb_lineno)
    print(str(e))

Script completed successfully.


In [19]:
layer_name = "Service Area"

In [20]:
impedance = "Biking by distance"

In [34]:
dist = 17582.4

In [23]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/network-analyst/make-service-area-layer.htm
#Create a new service area layer.
result_object = arcpy.na.MakeServiceAreaAnalysisLayer(inNetworkDataset, layer_name, impedance, 
                                                      "FROM_FACILITIES", [dist], None, "LOCAL_TIME_AT_LOCATIONS", 
                                                      "POLYGONS", "STANDARD", "OVERLAP", "RINGS", "100 Meters", None, None)

In [24]:
#Get the layer object from the result object. The service area layer can now be referenced using the layer object.
layer_object = result_object.getOutput(0)

In [25]:
#Get the names of all the sublayers within the service area layer.
sublayer_names = arcpy.na.GetNAClassNames(layer_object)

In [26]:
sublayer_names

{'Facilities': 'Facilities',
 'Barriers': 'Point Barriers',
 'SALines': 'Lines',
 'PolylineBarriers': 'Line Barriers',
 'SAPolygons': 'Polygons',
 'PolygonBarriers': 'Polygon Barriers'}

In [27]:
#Stores the layer names that we will use later
facilities_layer_name = sublayer_names["Facilities"]
polygons_layer_name = sublayer_names["SAPolygons"]

In [12]:
input_folder = r"T:\MPO\RTP\FY20 2045 Update\Data and Resources"

In [13]:
facilities = os.path.join(input_folder, "PerformanceAnalysis", "service_transit_equity", "service_stops.shp")

In [28]:
#Load the fire stations as facilities.
arcpy.na.AddLocations(layer_object, facilities_layer_name, facilities,
                        field_mappings, "", exclude_restricted_elements = "EXCLUDE")

<Result 'GPLYR_{60EA219F-E961-4462-B747-EED0CAE81234}'>

In [29]:
#Get sublayers to work with later
facilities_sublayer = layer_object.listLayers(facilities_layer_name)[0]
polygons_sublayer = layer_object.listLayers(polygons_layer_name)[0]

In [30]:
#Get the Service Area Layer's solver properties. This can be used to set individual properties later without 
#re-creating the layer.
solver_properties = arcpy.na.GetSolverProperties(layer_object)

In [31]:
#Solve the service area layer
arcpy.na.Solve(layer_object)

id,value
0,a Layer object
1,true


In [32]:
#Transfer the FireStationID field from the input Facilities to the output Polygons
arcpy.management.AddJoin(polygons_sublayer, "FacilityID", facilities_sublayer, "ObjectID")

<Result 'GPLYR_{E777C30A-9913-4619-B0F9-9B7FAADC45B8}'>

In [33]:
#The joined fields are qualified by the feature class name of the joined table, so determine the feature class names
field_qualifier_pol = os.path.basename(polygons_sublayer.dataSource)
target_field_name = "%s.name" % field_qualifier_pol
field_qualifier_fac = os.path.basename(facilities_sublayer.dataSource)
expression = "!%s.name!" % field_qualifier_fac
arcpy.management.CalculateField(polygons_sublayer, target_field_name,
                                expression, "PYTHON")
arcpy.management.RemoveJoin(polygons_sublayer)

<Result 'GPLYR_{E777C30A-9913-4619-B0F9-9B7FAADC45B8}'>

In [35]:
out_featureclass = "ServiceArea"

In [37]:
mode = 'Biking'

In [38]:
#Append the polygons to the output feature class. If this was the first solve, create the feature class.
if not arcpy.Exists(out_featureclass):
    arcpy.management.CopyFeatures(polygons_sublayer, out_featureclass + mode)
else:
    arcpy.management.Append(polygons_sublayer, out_featureclass)